In [ ]:
from batchgenerators.utilities.file_and_folder_operations import *
import cv2
import os
from skimage.filters import threshold_otsu
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json

Setting up nnU-Net folders

In [ ]:
ct_dir:str = '/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/CAT_scans_Preprocessed',
nnunet_dataset_id: int = 250
task_name = "Skull_2024"

foldername = "Dataset%03.0d_%s" % (nnunet_dataset_id, task_name)
nnUNet_raw = "/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/nnunet_data/nnUNet_raw"
nnUNet_preprocessed = "/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/nnunet_data/nnUNet_preprocessed"
out_base = join(nnUNet_raw, foldername)

Generate .json file

In [ ]:
generate_dataset_json(out_base,
                      channel_names={0: "CT"},
                      labels={
                          "background": 0,
                          "skull": 1
                          },
                        num_training_cases=576, 
                        file_ending='.png',
                        dataset_name=task_name, 
                        reference='https://github.com/BYO-UPM/Craneal_CT',
                        release='release',
                        description=task_name)

Get the binary masks: range (0, 1)

In [ ]:
input_dir = '/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/Dataset/Labeled Data PNG/External Dataset/P13/Manual Mask'
ind = [2, 3, 4]
for i in ind:
    output_dir = f'/home/ysun@gaps_domain.ssr.upm.es/Craneal_CT/U-Mamba/data/nnUNet_raw/Dataset36{i}_Skull/labelsTr'
    counter = 1
    for filename in sorted(os.listdir(input_dir)):
        if filename.endswith('.png'):
            # Open the image
            img_path = os.path.join(input_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
            # Apply threshold to create binary mask
            threshold1 = threshold_otsu(img)
            binary_image = img > threshold1
            binary_image=binary_image.astype(int)
                
            # Save the binary image
            new_filename = f"10_{counter:02d}.png"
            counter += 1
            output_path = os.path.join(output_dir, new_filename)
            cv2.imwrite(output_path, binary_image)

Change all the CT images into 1 channel grey-scale images

In [ ]:
def convert_to_grayscale_cv2(source_directory, destination_directory):
    # Initialize a counter for the filenames
    counter = 1

    # Iterate over all files in the source directory
    for filename in sorted(os.listdir(source_directory)):
        if filename.endswith(".png"):
            # Read the image in grayscale mode
            img = cv2.imread(os.path.join(source_directory, filename), cv2.IMREAD_GRAYSCALE)
            # Save the grayscale image to the destination directory
            new_filename = f"10_{counter:02d}_0000.png"
            counter += 1
            cv2.imwrite(os.path.join(destination_directory, new_filename), img)
            
            # Verify the image has 1 channel
            if len(img.shape) == 2:
                print(f"{filename} is successfully converted to grayscale with 1 channel.")
            else:
                print(f"Error: {filename} still has {img.shape[-1]} channels.")
            
            print(f"Converted {filename} to grayscale and saved.")

source_dir = "/home/Craneal_CT/Codes/nnUNet/nnunet_data/nnUNet_raw/Dataset250_Skull_2024/imagesTr_RGB"
dest_dir = "/home/Craneal_CT/Codes/nnUNet/nnunet_data/nnUNet_raw/Dataset250_Skull_2024/imagesTr"
convert_to_grayscale_cv2(source_dir, dest_dir)